In [9]:
import os
import json
import requests
from dotenv import load_dotenv

In [10]:
# Load API key from .env file
load_dotenv()
API_KEY = os.getenv("SPORTSRADAR_API_KEY")

if not API_KEY:
    raise ValueError("❌ API Key not found in .env file")


In [11]:
# Read game IDs from series_game_ids.json
year = '2024'
# PRE IST PST REG PIT
season_type = 'REG'
file_path = f'../../data/nba/api_data/game_ids/{year}_{season_type}_season_ids.json'
with open(file_path, "r") as f:
    series_games = json.load(f)

In [12]:
def fetch_game_details(game_id, api_key):
    url = f"https://api.sportradar.com/nba/production/v8/en/games/{game_id}/summary.json"
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

In [13]:
os.makedirs(f'../../data/nba/api_data/match_stats/{year}',exist_ok=True)

In [17]:
all_game_ids = series_games
failed_game_ids = []
total_ids = len(all_game_ids)
print(f"📊 Total games to fetch: {total_ids}")
success, fail = 0, 0

for idx, game_id in enumerate(all_game_ids, start=1):
        if success != 777:
             success+=1
             continue
        try:
            data = fetch_game_details(game_id, API_KEY)

            # Save file as games/<game_id>.json
            file_path = os.path.join(f"../../data/nba/api_data/match_stats/{year}/{season_type}_{game_id}.json")
            with open(file_path, "w") as f:
                json.dump(data, f, indent=4)

            success += 1
            print(f"✅ {idx}/{total_ids} Saved {game_id}.json (Success: {success}, Fail: {fail})")

        except Exception as e:
            fail += 1
            failed_game_ids.append(game_id)
            print(f"❌ {idx}/{total_ids} Failed {game_id}: {e} (Success: {success}, Fail: {fail})")

print(f"\n🎯 Finished! {success} succeeded, {fail} failed out of {total_ids}")


📊 Total games to fetch: 1241
✅ 778/1241 Saved 95fcfa1d-297a-4b99-8112-fbca6f9265a3.json (Success: 778, Fail: 0)

🎯 Finished! 1241 succeeded, 0 failed out of 1241


In [15]:
failed_game_ids_file_path = f'../../data/nba/api_data/failed_game_ids/{year}_{season_type}_season_ids.json'
with open(failed_game_ids_file_path, "w") as f:
    json.dump(failed_game_ids, f, indent=4)